<a href="https://colab.research.google.com/github/dev129/pulse-next-band/blob/main/PPG_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 49.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 3.0.1 which is incompatible.
gradio 5.50.0 requires pandas<3.0,>=1.0, but you have pandas 3.0.1 which is incompatible.
db-dtypes 1.5.0 requires pandas<3.0.0,>=1.5.3, but you have pandas 3.0.1 which is incompatible.
bqplot 0.12.45 requires pandas<3.0.0,>=1.0.0, but you have pandas 3.0.1 which is incompatible.


In [2]:
import wfdb
import matplotlib.pyplot as plt

In [ ]:
# Load the record you already loaded
record = wfdb.rdrecord('mitdb/100', pn_dir='mitdb')

# Just plot first 1000 points of channel 0
signal = record.p_signal[:1000, 0]

In [ ]:
print(record.p_signal.shape)
print(record.sig_name)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(signal)
plt.title("ECG Signal - First 1000 samples")
plt.xlabel("Sample number")
plt.ylabel("Amplitude (mV)")
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
print(np.max(signal))        # highest value overall
print(np.argmax(signal))     # WHERE is that highest value?

In [ ]:
differences = np.diff(signal)  # calculates signal[i] - signal[i-1] for all i

print("Max difference:", np.max(differences))
print("Min difference:", np.min(differences))
print("Mean difference:", np.mean(differences))
print("Std difference:", np.std(differences))

In [ ]:
# signal = record.p_signal[:1000, 0]
# differences = np.diff(signal) # calculates signal[i] - signal[i-1] for all i
#
# results = {}
#
# for threshold in np.arange(0.107, 0.30, 0.01):
#     spike_locations = []
#     for i in range(len(differences)):
#         if differences[i] > threshold:
#             spike_locations.append(i)  # should this be i or i+1? WHY?
#     results[round(threshold, 3)] = len(spike_locations)
#
# print(results)

In [ ]:
# signal = record.p_signal[:1000, 0]
# differences = np.diff(signal)
#
# threshold = 0.257  # YOU decide based on mean + std logic above
#
# spike_locations = []
#
# for i in range(len(differences)):
#     if differences[i] > threshold:
#         spike_locations.append(i)
#
# print(spike_locations)
# print("Number of spikes found:", len(spike_locations))

In [ ]:
from scipy.signal import find_peaks

signal = record.p_signal[:1000, 0]
peaks, _ = find_peaks(signal, height=0.5, distance=200)
print("Peak locations:", peaks)
print("Peak values:", signal[peaks])

# NOW YOU WRITE THIS PART:
# Calculate differences between consecutive peaks
# What numpy function gives you differences between array elements?

In [ ]:
rr_intervals=np.diff(peaks)
rr_intervals

In [ ]:
rr_seconds=rr_intervals/360
rr_seconds

In [ ]:
HRV=np.std(rr_intervals)
HRV

In [ ]:
heart_rate=60/rr_seconds
heart_rate

In [ ]:
features = {
    'mean_hr'    : np.mean(heart_rate),
    'hrv_std'    : np.std(rr_intervals),
    'mean_rr'    : np.mean(rr_intervals),
    'min_rr'     : np.min(rr_intervals),
    'max_rr'     : np.max(rr_intervals),
    'rr_range'   : np.max(rr_intervals) - np.min(rr_intervals)
}
print(features)

In [ ]:
annotation = wfdb.rdann('100', 'atr',pn_dir='mitdb')  # no pn_dir!

In [ ]:
print(annotation.sample[:20])
print(annotation.symbol[:20])

In [ ]:
# Count how many of each label exists in patient 100
import collections

all_annotations = wfdb.rdann('100', 'atr', pn_dir='mitdb')
label_counts = collections.Counter(all_annotations.symbol)
print(label_counts)

In [ ]:
import numpy as np
from scipy.signal import find_peaks
import pandas as pd
import wfdb

def extract_features(record_name):
    # Load signal
    record = wfdb.rdrecord(record_name, pn_dir='mitdb')
    signal = record.p_signal[:, 0]

    # Load annotations
    ann = wfdb.rdann(record_name, 'atr', pn_dir='mitdb')
    ann_symbol = np.array(ann.symbol)

    rows = []
    window_size = 1000

    for start in range(0, len(signal)-window_size, window_size):
        window = signal[start:start+window_size]

        # Find peaks
        peaks, _ = find_peaks(window, height=0.5, distance=200)

        if len(peaks) < 2:
            continue

        rr_intervals = np.diff(peaks)
        rr_seconds=rr_intervals/360
        hr=60/rr_seconds
        # YOU WRITE THESE:
        mean_hr = np.mean(hr)
        hrv_std = np.std(rr_intervals)
        mean_rr = np.mean(rr_intervals)

        # Find label for this window
        # which annotations fall in this window?
        mask = (ann.sample >= start) & (ann.sample < start+window_size)
        window_labels = ann_symbol[mask]

        # Most common label in window
        if len(window_labels) == 0:
            continue
        if 'A' in window_labels or 'V' in window_labels:
            label = 'ABNORMAL'
        else:
            label = 'N'

        rows.append({
            'mean_hr': mean_hr,
            'hrv_std': hrv_std,
            'mean_rr': mean_rr,
            'label': label
        })

    return pd.DataFrame(rows)

In [ ]:
df = extract_features('100')
print(df.head())
print(df['label'].value_counts())

In [ ]:
print("RR intervals:", rr_intervals)
print("RR seconds:", rr_seconds)
print("HR values:", 60/rr_seconds)

In [ ]:
records = wfdb.get_record_list('mitdb')
print(records)
print(len(records))

In [ ]:
all_dfs = []
for record in records:
    try:
        df = extract_features(record)
        df['patient'] = record  # track which patient
        all_dfs.append(df)
    except:
        print(f"Skipped: {record}")x

final_df = pd.concat(all_dfs, ignore_index=True)
print(final_df.shape)
print(final_df['label'].value_counts())